In [603]:
# !pip install textblob 
# !pip install scikit-learn
# !pip install gensim
# !pip install nltk
# !pip install seaborn
# !pip install matplotlib
# !pip install skfeature-chappers



In [350]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import model_selection
from textblob import TextBlob 
import numpy as np
import re
import nltk
nltk.download('stopwords')

from gensim.parsing.porter import PorterStemmer
from gensim.utils import tokenize

from nltk.corpus import stopwords
stopwords = [word for word in stopwords.words('english')]
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/suleman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [351]:

def listing():
    """
        i) Read data from csv.
        ii) Convert data to numpy array.
        iii) Split data with comma.
        iv) Appends data into list.
    """

    url = "Data/all_brands_export.csv"
    # url = "Data/acer-phones_export.csv"
    db=pd.read_csv(url,low_memory = False)#,sep= ',', error_bad_lines=False, index_col=False, dtype='unicode')
    # db["review"]
    reviews = np.array(db["review"])
    # print(reviews)

    super_list = []
    for sent in reviews:

        temp= []
        try:
            temp = list(sent.split(","))
            super_list.append(temp)

        except:
            super_list.append("None")
    return super_list


In [352]:
def pre_process(sentence):  
    """
        i) Remove special characters.
        ii) Remove single characters.
        iii) Replace multiple spaces with single spaces.
    """


    # Remove all the special characters
    processed_sentence = re.sub(r'\W', ' ', str(sentence))

    # remove all single characters
    processed_sentence = re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_sentence)

    # Remove single characters from the start
    processed_sentence= re.sub(r'\^[a-zA-Z]\s+', ' ', processed_sentence) 

    # Substituting multiple spaces with single space
    processed_sentence= re.sub(r'\s+', ' ', processed_sentence, flags=re.I)

    # Removing prefixed 'b'
    processed_sentence = re.sub(r'^b\s+', '', processed_sentence)
    
   
    return processed_sentence


In [353]:
def updated_sentence(sentence):
    """
        i) Lower cases the sentence.
        ii) Sentence stemming.
        iii) Tokenization of sentence.
        iv) Stop-words removal.
        v) Returns list after all above operations.
    """
    sentence = sentence.lower()
    new = " "
    p = PorterStemmer()
    processed = pre_process(sentence)
    stemmed = p.stem(processed)
    tokens = tokenize(stemmed.strip())
    lst=list(tokens)
    lst2 = []
    for i in lst:
        #using nltk dic for stopwords
        if  i not in stopwords :
            lst2.append(i)
    return (new.join(lst2))

In [354]:
def polarity_scaling(polarity):
    """
        Scaling of polarity into three classes i.e. negative, positive and neutral.
    """
    # 0  nutral
    if polarity  == 0:
        return 0
    # greater then 0  positive
    elif polarity > 0:
        return 1
    else:
    # less then 0  negative
        return -1    

In [355]:

def reviews_list_polarity(super_list):
    p = PorterStemmer()

    features_dictionary={
        "battery": ["cell","power","battery","charge","recharge","cycle","volt","ampere","current","electric"],
        "display": ["display","screen","panel","brightness","glass","touch","graphics","resolution","refresh rate","color","colour"],
        "camera": ["camera","pixels","image","picture","zoom","selfie","optical","flash","record","hdr","panorama","night vision","time lapse","slow motion","slo-mo","fps","video"],
        "storage": ["storage","ram","gb","mb","tb","space","memory","cache"],
        # "price": ["price","cost","value","money","expensive","cheap","affordable"]
    }
    
    
    battery_polarity =[]
    display_polarity =[]
    camera_polarity =[]
    storage_polarity =[]
    # price_polarity =[]
    
    
    
    for_reviews = []
    for_polarity = []
    updated_review_Dict = {}
    super_list_2 = []
    list_reviews = []
    list_polarity = []

    for remove_none_type in super_list:
        if remove_none_type != "None":
            super_list_2.append(remove_none_type)

            lst_list = []
            lst_list_2 = []
            for i in super_list_2:
                new1 =i.strip().split(".")

            for j in new1:
                j = updated_sentence(j)
                new_sentence =j.replace("\r\n" ,"")
                for feature in features_dictionary:
                    for word in features_dictionary[feature]:
                        if p.stem(word) in new_sentence:
                        # if word in new_sentence:
                            pol = TextBlob(new_sentence).sentiment.polarity
                            polarity=polarity_scaling(pol)
                            tup = (new_sentence ,polarity)
                            lst_list.append(tup)

                            if  features_dictionary[feature] == features_dictionary["display"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                display_polarity.append(polarity)
                            # else:
                            #     camera_polarity.append(0)
                            if  features_dictionary[feature] == features_dictionary["camera"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                camera_polarity.append(polarity)
                            # else:
                            #     camera_polarity.append(0)

                            if  features_dictionary[feature] == features_dictionary["battery"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                battery_polarity.append(polarity)
                            # else:
                            #     battery_polarity.append(0)

                            if  features_dictionary[feature] == features_dictionary["storage"]:
                                pol = TextBlob(new_sentence).sentiment.polarity
                                polarity=polarity_scaling(pol)
                                # tup = (new_sentence ,polarity)
                                # lst_list.append(tup)
                                storage_polarity.append(polarity)
                            # else:
                            #     storage_polarity.append(0)
    
            for remove_tuple in lst_list:
                if remove_tuple != ('',0):
                    lst_list_2.append(remove_tuple)

            if lst_list_2 != []:
                for_reviews.append(lst_list_2[0][0])
                for_polarity.append(lst_list_2[0][1])
  
        updated_review_Dict["reviews"]=for_reviews
        updated_review_Dict["polarity"]=for_polarity

        
        list_reviews=updated_review_Dict["reviews"]
        list_polarity=updated_review_Dict["polarity"]
    return list_reviews,list_polarity ,battery_polarity ,display_polarity ,storage_polarity ,camera_polarity


In [356]:
def per_mobile_phone_review():
    review_list = listing()
    updated_review_list=[]
    for i in range(len(review_list)):
        updated_review_list.append(reviews_list_polarity(review_list[i]))
    return updated_review_list

In [357]:
data=pd.DataFrame(per_mobile_phone_review())
data.rename(columns = {0:'reviews', 1:'polarity',2:'battery_polarity',
                       3:'display_polarity',4:'storage_polarity',5:'camera_polarity'},
            inplace = True)
data
for i in data["polarity"] :
    if i == []:
        i.append(0)
for i in data["battery_polarity"] :
    if i == []:
        i.append(0)
for i in data["display_polarity"] :
    if i == []:
        i.append(0)
        
for i in data["storage_polarity"] :
    if i == []:
        i.append(0)
for i in data["camera_polarity"] :
    if i == []:
        i.append(0)


In [639]:
url = "Data/all_brands_export.csv"
    # url = "Data/acer-phones_export.csv"
db=pd.read_csv(url,low_memory = False,sep =',')
db

,Link,Name,Weight,OS,Storage,Hits,Screen_size,Screen_resolution,RAM,SOC,...,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82
0,https://www.gsmarena.com/acer_chromebook_tab_1...,Acer Chromebook Tab 10,544.3g 9.9mm thickness,Chrome OS,32GB storage microSDXC,313 765 hits,"9.7""",1536x2048 pixels,4 GB RAM,Rockchip RK3399,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.gsmarena.com/acer_iconia_talk_s-83...,Acer Iconia Talk S,260g 9.4mm thickness,Android 6.0,16GB/32GB storage microSDXC,522 488 hits,"7.0""",720x1280 pixels,2 GB RAM,MT8735,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.gsmarena.com/acer_liquid_z6_plus-8...,Acer Liquid Z6 Plus,169g 8.5mm thickness,Android 6.0,32GB storage microSDXC,680 638 hits,"5.5""",1080x1920 pixels,3 GB RAM,MT6753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.gsmarena.com/acer_liquid_z6-8304.php,Acer Liquid Z6,126g 8.5mm thickness,Android 6.0,8GB storage microSDXC,266 266 hits,"5.0""",720x1280 pixels,1 GB RAM,MT6737,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://www.gsmarena.com/acer_iconia_tab_10_a3...,Acer Iconia Tab 10 A3-A40,8.9mm thickness,Android 6.0,16GB/32GB/64GB storage microSDXC,357 820 hits,"10.1""",1920x1200 pixels,2 GB RAM,MT8163A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10924,https://www.gsmarena.com/zte_f103-3099.php,ZTE F103,90g 12.6mm thickness,Feature phone,microSDHC slot,83 571 hits,"2.0""",176x220 pixels,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10925,https://www.gsmarena.com/zte_f101-3101.php,ZTE F101,90g 12.6mm thickness,Feature phone,microSDHC slot,125 180 hits,"2.0""",176x220 pixels,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10926,https://www.gsmarena.com/zte_f100-3100.php,ZTE F100,90g 12.6mm thickness,Feature phone,microSDHC slot,120 275 hits,"2.0""",176x220 pixels,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10927,https://www.gsmarena.com/zte_coral200_sollar-3...,ZTE Coral200 Sollar,85g 18.1mm thickness,Feature phone,4MB storage no card slot,104 754 hits,NaN,128x128 pixels,,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [633]:
new_Data = pd.DataFrame()
new_Data["Link"] = db["Link"]
# new_Data["Name"] = db["Name"]
new_Data["battery_polarity"]=data["battery_polarity"]
new_Data["display_polarity"]=data["display_polarity"]
new_Data["storage_polarity"]=data["storage_polarity"]
new_Data["camera_polarity"]=data["camera_polarity"]

In [634]:
new_Data.head()

,Link,battery_polarity,display_polarity,storage_polarity,camera_polarity
0,https://www.gsmarena.com/acer_chromebook_tab_1...,[0],"[1, 0, 0, -1, 0, 0]","[0, -1, -1]",[0]
1,https://www.gsmarena.com/acer_iconia_talk_s-83...,"[1, 1]",[1],"[1, 1, 1, 1]","[-1, -1, -1, -1, 1, 0]"
2,https://www.gsmarena.com/acer_liquid_z6_plus-8...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-1, -1, 1, 1, 0, 1]","[1, 0]","[-1, 0]"
3,https://www.gsmarena.com/acer_liquid_z6-8304.php,[1],"[0, 0, 0, 1, 1]","[1, 1]",[0]
4,https://www.gsmarena.com/acer_iconia_tab_10_a3...,"[1, 1]","[1, 1, 1]","[1, 1, 1, 0]",[0]


In [635]:
#using averg

def features_sum():
    battery_polarity_sum = []
    storage_polarity_sum = []
    display_polarity_sum = []
    camera_polarity_sum = []
    for battery_sum in new_Data["battery_polarity"]:
        # print(sum(battery_sum))
        add=sum(battery_sum)
        res = "{:.2f}".format(add)
        battery_polarity_sum.append(res)
    for display_sum in new_Data["display_polarity"]:
        # print(sum(display_sum))
        add=sum(display_sum)
        res = "{:.2f}".format(add)
    
        display_polarity_sum.append(res)
        
    for storage_sum in new_Data["storage_polarity"]:
        # print(sum(storage_sum))
        add=sum(storage_sum)
        res = "{:.2f}".format(add)
        storage_polarity_sum.append(res)
        
    for camera_sum in new_Data["camera_polarity"]:
        # print(sum(camera_sum))
        add=sum(camera_sum)
        res = "{:.2f}".format(add)
        camera_polarity_sum.append(res)
        
    new_Data["battery_sum"] = battery_polarity_sum    
    new_Data["display_sum"] =display_polarity_sum    
    new_Data["storage_sum"] = storage_polarity_sum    
    new_Data["camera_sum"] = camera_polarity_sum   
    
    # return battery_polarity_sum,display_polarity_sum,storage_polarity_sum,camera_polarity_sum
        

In [636]:
features_sum()

In [637]:
new_Data.head()

,Link,battery_polarity,display_polarity,storage_polarity,camera_polarity,battery_sum,display_sum,storage_sum,camera_sum
0,https://www.gsmarena.com/acer_chromebook_tab_1...,[0],"[1, 0, 0, -1, 0, 0]","[0, -1, -1]",[0],0.00,0.00,-2.00,0.00
1,https://www.gsmarena.com/acer_iconia_talk_s-83...,"[1, 1]",[1],"[1, 1, 1, 1]","[-1, -1, -1, -1, 1, 0]",2.00,1.00,4.00,-3.00
2,https://www.gsmarena.com/acer_liquid_z6_plus-8...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-1, -1, 1, 1, 0, 1]","[1, 0]","[-1, 0]",0.00,1.00,1.00,-1.00
3,https://www.gsmarena.com/acer_liquid_z6-8304.php,[1],"[0, 0, 0, 1, 1]","[1, 1]",[0],1.00,2.00,2.00,0.00
4,https://www.gsmarena.com/acer_iconia_tab_10_a3...,"[1, 1]","[1, 1, 1]","[1, 1, 1, 0]",[0],2.00,3.00,3.00,0.00


In [640]:
new_Data.to_csv('features_polarity_sum.csv', mode='w', index=False)

# # print message
print("Data appended successfully.")

Data appended successfully.


In [641]:
url = "features_polarity_sum.csv"
polarity_sum=pd.read_csv(url,low_memory = False)
polarity_sum.head()

,Link,battery_polarity,display_polarity,storage_polarity,camera_polarity,battery_sum,display_sum,storage_sum,camera_sum
0,https://www.gsmarena.com/acer_chromebook_tab_1...,[0],"[1, 0, 0, -1, 0, 0]","[0, -1, -1]",[0],0.0,0.0,-2.0,0.0
1,https://www.gsmarena.com/acer_iconia_talk_s-83...,"[1, 1]",[1],"[1, 1, 1, 1]","[-1, -1, -1, -1, 1, 0]",2.0,1.0,4.0,-3.0
2,https://www.gsmarena.com/acer_liquid_z6_plus-8...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[-1, -1, 1, 1, 0, 1]","[1, 0]","[-1, 0]",0.0,1.0,1.0,-1.0
3,https://www.gsmarena.com/acer_liquid_z6-8304.php,[1],"[0, 0, 0, 1, 1]","[1, 1]",[0],1.0,2.0,2.0,0.0
4,https://www.gsmarena.com/acer_iconia_tab_10_a3...,"[1, 1]","[1, 1, 1]","[1, 1, 1, 0]",[0],2.0,3.0,3.0,0.0


In [604]:
# def features_sum():
#     battery_polarity_sum = []
#     storage_polarity_sum = []
#     display_polarity_sum = []
#     camera_polarity_sum = []
#     for battery_sum in new_Data["battery_polarity"]:
#         num = max(battery_sum, key=lambda battery:battery_sum.count(battery))
#         battery_polarity_sum.append(num)
#     for display_sum in new_Data["display_polarity"]:
#         num = max(display_sum, key=lambda display:display_sum.count(display))
#         display_polarity_sum.append(num)
#     for storage_sum in new_Data["storage_polarity"]:
#         num = max(storage_sum, key=lambda storage:storage_sum.count(storage))
#         storage_polarity_sum.append(num)
#     for camera_sum in new_Data["camera_polarity"]:
#         num = max(camera_sum, key=lambda camera:camera_sum.count(camera))
#         camera_polarity_sum.append(num)
    
#     new_Data["battery_overall"] = battery_polarity_sum    
#     new_Data["display_overall"] =display_polarity_sum    
#     new_Data["storage_overall"] = storage_polarity_sum    
#     new_Data["camera_overall"] = camera_polarity_sum 
    
#     # return battery_polarity_sum,display_polarity_sum,storage_polarity_sum,camera_polarity_sum